# HSE 2022: Mathematical Methods for Data Analysis

## Homework 2

In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings("ignore")

%matplotlib inline

sns.set(style="darkgrid")

### Data

For this homework we use Dataset from seaborn on diamonds prices.

In [2]:
data = sns.load_dataset('diamonds')

y = data.price
x = data.drop(['price'], axis=1)
columns = data.drop(['price'], axis=1).columns

## Linear regression

#### 0. [0.25 points] Encode categorical variables.

In [3]:
from sklearn.preprocessing import OrdinalEncoder

display(data)

encoder = OrdinalEncoder()
data_ordinal = pd.DataFrame(encoder.fit_transform(data),
                            columns=data.columns)
features = data_ordinal.drop('price', axis=1)
target = data_ordinal['price']

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


#### 1. [0.25 points] Split the data into train and test sets with ratio 80:20 with random_state=17.

In [4]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2,
                                                                            random_state=17)

#### 2. [1 point] Train models on train data using StatsModels library and apply it to the test set; use $RMSE$ and $R^2$ as the quality measure.

* [`LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html);
* [`Ridge`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) with $\alpha = 0.01$;
* [`Lasso`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) with $\alpha = 0.01$
* [`ElasticNet`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) with $\alpha = 0.01$, $l_{1}$_$ratio = 0.6$

Don't forget to scale the data before training the models with StandardScaler!

In [5]:
scaler = StandardScaler()
numerics = ['carat', 'depth', 'table', 'x', 'y', 'z']
scaler.fit(features_train[numerics])
features_train[numerics] = scaler.transform(features_train[numerics])
features_test[numerics] = scaler.transform(features_test[numerics])

In [6]:
features_train = sm.add_constant(features_train)
features_test = sm.add_constant(features_test)

linear_model = sm.OLS(target_train, features_train)
linear_results = linear_model.fit()
predicted_test = linear_results.predict(features_test)
mse = mean_squared_error(y_true=target_test, y_pred=predicted_test)
rmse = mse ** .5
print("MSE =", mse)
print("RMSE =", rmse)
print("R2 = ", r2_score(y_true=target_test, y_pred=predicted_test))

MSE = 776022.1336650986
RMSE = 880.9211847067243
R2 =  0.9148058965444855


In [7]:
ridge_model = linear_model.fit_regularized(L1_wt=0, alpha=0.01)
ridge_result = sm.regression.linear_model.OLSResults(linear_model, ridge_model.params,
                                                     linear_model.normalized_cov_params)
predicted_ridge = ridge_model.predict(features_test)
mse = mean_squared_error(y_true=target_test, y_pred=predicted_test)
rmse = mse ** .5
print("MSE =", mse)
print("RMSE =", rmse)
print('R2 = ', r2_score(y_true=target_test, y_pred=predicted_ridge))

MSE = 776022.1336650986
RMSE = 880.9211847067243
R2 =  0.9125959534506798


In [8]:
lasso_model = linear_model.fit_regularized(L1_wt=1, alpha=0.01)
lasso_result = sm.regression.linear_model.OLSResults(linear_model, lasso_model.params,
                                                     linear_model.normalized_cov_params)
predicted_lasso = lasso_model.predict(features_test)
mse = mean_squared_error(y_true=target_test, y_pred=predicted_lasso)
rmse = mse ** .5
print("MSE =", mse)
print("RMSE =", rmse)
print('R2 = ', r2_score(y_true=target_test, y_pred=predicted_lasso))

MSE = 777573.0826038037
RMSE = 881.8010447962758
R2 =  0.9146356285861295


In [9]:
elastic_model = linear_model.fit_regularized(L1_wt=0.6, alpha=0.01)
elastic_result = sm.regression.linear_model.OLSResults(linear_model, elastic_model.params,
                                                       linear_model.normalized_cov_params)
predicted_elastic = elastic_model.predict(features_test)
mse = mean_squared_error(y_true=target_test, y_pred=predicted_test)
rmse = mse ** .5
print("MSE =", mse)
print("RMSE =", rmse)
print('R2 = ', r2_score(y_true=target_test, y_pred=predicted_elastic))

MSE = 776022.1336650986
RMSE = 880.9211847067243
R2 =  0.9141784641073025


#### 3. [1 point] Explore the values of the parameters of the resulting models and compare the number of zero weights in them. Comment on the significance of the coefficients, overal model significance and other related factors from the results table

In [10]:
linear_results.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
====================================================================
Model:              OLS              Adj. R-squared:     0.914      
Dependent Variable: price            AIC:                707856.7639
Date:               2022-12-08 03:47 BIC:                707943.4887
No. Observations:   43152            Log-Likelihood:     -3.5392e+05
Df Model:           9                F-statistic:        5.078e+04  
Df Residuals:       43142            Prob (F-statistic): 0.00       
R-squared:          0.914            Scale:              7.7894e+05 
--------------------------------------------------------------------
             Coef.    Std.Err.    t     P>|t|    [0.025     0.975]  
--------------------------------------------------------------------
const       2624.2705  16.3907 160.1069 0.0000  2592.1444  2656.3967
carat       2699.0777  21.4817 125.6453 0.0000  2656.9731  2741.1823
cut           52.8693   4.2725  12.3744 0.0000    44.4951    61.2434
color       -190.5337   2.6168 -72.8112 0.0000  -195.6627  -185.4047
clarity      214.5259   2.5507  84.1037 0.0000   209.5264   219.5253
depth        -46.6299   9.4944  -4.9113 0.0000   -65.2391   -28.0208
table       -121.5332   4.6368 -26.2106 0.0000  -130.6214  -112.4450
x          -1372.6718  89.2151 -15.3861 0.0000 -1547.5351 -1197.8085
y           1893.3603  87.5187  21.6338 0.0000  1721.8220  2064.8987
z           -188.8980  66.9466  -2.8216 0.0048  -320.1145   -57.6814
--------------------------------------------------------------------
Omnibus:              7912.396      Durbin-Watson:         1.997    
Prob(Omnibus):        0.000         Jarque-Bera (JB):      51815.141
Skew:                 0.724         Prob(JB):              0.000    
Kurtosis:             8.169         Condition No.:         152      
====================================================================

"""

На уровне значимости 0.05 нет признаков имеющие нулевые веса.

In [11]:
linear_results.pvalues[linear_results.pvalues > 0.05].index.values

array([], dtype=object)

In [12]:
ridge_result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
====================================================================
Model:              OLS              Adj. R-squared:     0.911      
Dependent Variable: price            AIC:                709152.9296
Date:               2022-12-08 03:47 BIC:                709239.6544
No. Observations:   43152            Log-Likelihood:     -3.5457e+05
Df Model:           9                F-statistic:        4.913e+04  
Df Residuals:       43142            Prob (F-statistic): 0.00       
R-squared:          0.911            Scale:              8.0269e+05 
---------------------------------------------------------------------
            Coef.    Std.Err.     t      P>|t|     [0.025     0.975] 
---------------------------------------------------------------------
const     2238.4162   16.6388  134.5302  0.0000  2205.8039  2271.0285
carat     2165.6306   21.8068   99.3099  0.0000  2122.8888  2208.3723
cut        117.0268    4.3371   26.9825  0.0000   108.5260   125.5277
color     -161.3356    2.6564  -60.7343  0.0000  -166.5422  -156.1290
clarity    246.8583    2.5893   95.3368  0.0000   241.7832   251.9334
depth      -56.4293    9.6380   -5.8549  0.0000   -75.3201   -37.5386
table     -128.3472    4.7070  -27.2675  0.0000  -137.5730  -119.1215
x          169.0071   90.5651    1.8661  0.0620    -8.5022   346.5164
y          500.0043   88.8430    5.6280  0.0000   325.8702   674.1383
z          183.2733   67.9596    2.6968  0.0070    50.0712   316.4755
--------------------------------------------------------------------
Omnibus:              8038.335      Durbin-Watson:         1.996    
Prob(Omnibus):        0.000         Jarque-Bera (JB):      36234.753
Skew:                 0.849         Prob(JB):              0.000    
Kurtosis:             7.156         Condition No.:         152      
====================================================================

"""

На уровне значимости 0.05 признак ['x'] имеет нулевой вес.

In [13]:
features_train.columns[ridge_result.pvalues > 0.05].values

array(['x'], dtype=object)

In [14]:
lasso_result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
====================================================================
Model:              OLS              Adj. R-squared:     0.913      
Dependent Variable: price            AIC:                708261.0125
Date:               2022-12-08 03:47 BIC:                708347.7374
No. Observations:   43152            Log-Likelihood:     -3.5412e+05
Df Model:           9                F-statistic:        5.026e+04  
Df Residuals:       43142            Prob (F-statistic): 0.00       
R-squared:          0.913            Scale:              7.8627e+05 
---------------------------------------------------------------------
            Coef.    Std.Err.     t      P>|t|     [0.025     0.975] 
---------------------------------------------------------------------
const     2593.6833   16.4677  157.5013  0.0000  2561.4063  2625.9603
carat     2702.1952   21.5826  125.2026  0.0000  2659.8930  2744.4975
cut         62.5648    4.2925   14.5752  0.0000    54.1514    70.9783
color     -190.6321    2.6291  -72.5084  0.0000  -195.7852  -185.4790
clarity    216.1488    2.5627   84.3440  0.0000   211.1259   221.1718
depth      -59.4398    9.5390   -6.2313  0.0000   -78.1363   -40.7433
table     -134.7719    4.6586  -28.9299  0.0000  -143.9028  -125.6410
x          238.3825   89.6340    2.6595  0.0078    62.6982   414.0668
y          239.0742   87.9296    2.7189  0.0066    66.7305   411.4179
z         -145.8133   67.2609   -2.1679  0.0302  -277.6459   -13.9807
--------------------------------------------------------------------
Omnibus:              7859.417      Durbin-Watson:         1.996    
Prob(Omnibus):        0.000         Jarque-Bera (JB):      53481.710
Skew:                 0.706         Prob(JB):              0.000    
Kurtosis:             8.268         Condition No.:         152      
====================================================================

"""

На уровне значимости 0.05 нет нулевых признаков.

In [15]:
features_train.columns[lasso_result.pvalues > 0.05].values

array([], dtype=object)

In [16]:
elastic_result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
====================================================================
Model:              OLS              Adj. R-squared:     0.913      
Dependent Variable: price            AIC:                708461.6585
Date:               2022-12-08 03:47 BIC:                708548.3834
No. Observations:   43152            Log-Likelihood:     -3.5422e+05
Df Model:           9                F-statistic:        5.000e+04  
Df Residuals:       43142            Prob (F-statistic): 0.00       
R-squared:          0.913            Scale:              7.8994e+05 
---------------------------------------------------------------------
            Coef.    Std.Err.     t      P>|t|     [0.025     0.975] 
---------------------------------------------------------------------
const     2436.0075   16.5060  147.5830  0.0000  2403.6553  2468.3596
carat     2459.2494   21.6328  113.6814  0.0000  2416.8486  2501.6501
cut         87.3153    4.3025   20.2939  0.0000    78.8822    95.7483
color     -177.7663    2.6352  -67.4577  0.0000  -182.9314  -172.6012
clarity    229.5775    2.5687   89.3760  0.0000   224.5429   234.6122
depth      -45.6816    9.5612   -4.7778  0.0000   -64.4217   -26.9416
table     -132.9628    4.6694  -28.4753  0.0000  -142.1149  -123.8106
x          396.1678   89.8426    4.4096  0.0000   220.0746   572.2610
y          276.1604   88.1343    3.1334  0.0017   103.4155   448.9052
z         -103.6976   67.4174   -1.5381  0.1240  -235.8371    28.4418
--------------------------------------------------------------------
Omnibus:              7996.346      Durbin-Watson:         1.996    
Prob(Omnibus):        0.000         Jarque-Bera (JB):      44977.191
Skew:                 0.779         Prob(JB):              0.000    
Kurtosis:             7.753         Condition No.:         152      
====================================================================

"""

На уровне значимости 0.05 есть нулевой признак ['y'].

In [17]:
features_train.columns[elastic_result.pvalues > 0.05].values

array(['z'], dtype=object)

#### 4. [1 point] Implement one of the elimination algorithms that were described in the Seminar_4 (Elimination by P-value, Forward elimination, Backward elimination), make conclusions.

In [18]:
def elimination(model, alpha):
    if isinstance(model, sm.OLS):
        features_tmp = model.exog
        target_tmp = model.endog
        max_iter = model.exog.shape[1]

        dropping(alpha, features_tmp, max_iter, target_tmp)

        return sm.OLS(target_tmp, features_tmp)

    raise Exception('Not supported Model')


def dropping(alpha, features_tmp, max_iter, target_tmp):
    for i in range(max_iter):
        results_tmp = sm.OLS(target_tmp, features_tmp).fit()
        max_pvalue = results_tmp.pvalues.max()
        if max_pvalue >= alpha:
            col_to_drop = results_tmp.pvalues.index.values[results_tmp.pvalues.argmax()]
            features_tmp.drop(labels=col_to_drop, axis=1, level=0, inplace=True)
            continue
        break

In [19]:
threshold = 0.05
elimination_model = elimination(linear_model, threshold)
elim_results = elimination_model.fit()
elim_results.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
====================================================================
Model:              OLS              Adj. R-squared:     0.914      
Dependent Variable: y                AIC:                707856.7639
Date:               2022-12-08 03:47 BIC:                707943.4887
No. Observations:   43152            Log-Likelihood:     -3.5392e+05
Df Model:           9                F-statistic:        5.078e+04  
Df Residuals:       43142            Prob (F-statistic): 0.00       
R-squared:          0.914            Scale:              7.7894e+05 
---------------------------------------------------------------------
         Coef.     Std.Err.     t      P>|t|     [0.025      0.975]  
---------------------------------------------------------------------
const   2624.2705   16.3907  160.1069  0.0000   2592.1444   2656.3967
x1      2699.0777   21.4817  125.6453  0.0000   2656.9731   2741.1823
x2        52.8693    4.2725   12.3744  0.0000     44.4951     61.2434
x3      -190.5337    2.6168  -72.8112  0.0000   -195.6627   -185.4047
x4       214.5259    2.5507   84.1037  0.0000    209.5264    219.5253
x5       -46.6299    9.4944   -4.9113  0.0000    -65.2391    -28.0208
x6      -121.5332    4.6368  -26.2106  0.0000   -130.6214   -112.4450
x7     -1372.6718   89.2151  -15.3861  0.0000  -1547.5351  -1197.8085
x8      1893.3603   87.5187   21.6338  0.0000   1721.8220   2064.8987
x9      -188.8980   66.9466   -2.8216  0.0048   -320.1145    -57.6814
--------------------------------------------------------------------
Omnibus:              7912.396      Durbin-Watson:         1.997    
Prob(Omnibus):        0.000         Jarque-Bera (JB):      51815.141
Skew:                 0.724         Prob(JB):              0.000    
Kurtosis:             8.169         Condition No.:         152      
====================================================================

"""

#### 5. [1 point] Find the best (in terms of RMSE) $\alpha$ for Lasso regression using cross-validation with 4 folds. You must select values from range $[10^{-4}, 10^{3}]$.

In [20]:
folds = 4

In [21]:
%%time

alphas = np.logspace(-4, 3, 1000)
search = GridSearchCV(Ridge(), [{"alpha": alphas}], scoring="neg_mean_squared_error", cv=folds)
search.fit(features_train, target_train)
print(f"Best alpha = {search.best_params_['alpha']:.4f}")
# Best alpha = 2.0001
# CPU times: total: 5min 13s
# Wall time: 39.6 s
# Best alpha = 1.9111
# CPU times: total: 5min 23s
# Wall time: 51.3 s

Best alpha = 1.9111
CPU times: user 48.3 s, sys: 14.7 s, total: 1min 2s
Wall time: 18.6 s


## Gradient descent

#### 6. [3.5 points] Implement a Ridge regression model for the MSE loss function, trained by gradient descent.

All calculations must be vectorized, and python loops can only be used for gradient descent iterations. As a stop criterion, you must use (simultaneously):

* checking for the Absolute-value norm of the weight difference on two adjacent iterations (for example, less than some small number of the order of $10^{-6}$, set by the `tolerance` parameter);
* reaching the maximum number of iterations (for example, 10000, set by the `max_iter` parameter).

You need to implement:

* Full gradient descent:

$$
w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} Q(w_{k}).
$$

* Stochastic Gradient Descent:

$$
w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} q_{i_{k}}(w_{k}).
$$

$\nabla_{w} q_{i_{k}}(w_{k}) \, $ is the estimate of the gradient over the batch of objects selected randomly.

* Momentum method:

$$
h_0 = 0, \\
h_{k + 1} = \alpha h_{k} + \eta_k \nabla_{w} Q(w_{k}), \\
w_{k + 1} = w_{k} - h_{k + 1}.
$$

* Adagrad method:

$$
G_0 = 0, \\
G_{k + 1} = G_{k} + (\nabla_{w} Q(w_{k+1}))^2, \\
w_{k + 1} = w_{k} - \eta * \frac{\nabla_{w} Q(w_{k+1})}{\sqrt{G_{k+1} + \epsilon}}.
$$



To make sure that the optimization process really converges, we will use the `loss_history` class attribute. After calling the `fit` method, it should contain the values of the loss function for all iterations, starting from the first one (before the first step on the anti-gradient).

You need to initialize the weights with a random vector from normal distribution. The following is a template class that needs to contain the code implementing all variations of the models.

In [22]:
from sklearn.base import BaseEstimator


class LinReg(BaseEstimator):
    def __init__(self, delta=1.0, gd_type='Momentum',
                 tolerance=1e-4, max_iter=1000, w0=None, eta=1e-2, alpha=1e-3):
        """
        gd_type: str
            'GradientDescent', 'StochasticDescent', 'Momentum', 'Adagrad'
        delta: float
            proportion of object in a batch (for stochastic GD)
        tolerance: float
            for stopping gradient descent
        max_iter: int
            maximum number of steps in gradient descent
        w0: np.array of shape (d)
            init weights
        eta: float
            learning rate
        alpha: float
            momentum coefficient
        reg_cf: float
            regularization coefficient
        epsilon: float
            numerical stability
        """
        np.random.seed(10)
        self.delta = delta
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.eta = eta
        self.loss_history = None  # list of loss function values at each training iteration

    def fit(self, X, y):
        """
        X: np.array of shape (l, d)
        y: np.array of shape (l)
        ---
        output: self
        """

        supported_types = ['Momentum', 'GradientDescent', 'StochasticDescent']

        if self.gd_type in supported_types:
            self.loss_history = []
            if self.w0 is None:
                self.w0 = np.zeros(X.shape[1])
            self.w = np.array(self.w0)
            cur_w = np.array(self.w)
            h = np.zeros(X.shape[1])

            for i in range(0, self.max_iter):
                if self.gd_type == supported_types[1]:
                    self.w -= self.gradient_descent(X, y)
                elif self.gd_type == supported_types[2]:
                    self.stochastic_descent(X, y)
                else:
                    self.momentum_descent(X, h, y)
                self.loss_history.append(self.calc_loss(X, y))
                if np.linalg.norm(self.w - cur_w) < self.tolerance:
                    break
                cur_w = np.array(self.w)

            return self
        raise Exception('Unknown type')

    def momentum_descent(self, X, h, y):
        indexes = np.random.choice(X.shape[0], int(X.shape[0] * self.delta))
        h = self.alpha * h + self.gradient_descent(np.take(X, indexes, axis=0), np.take(y, indexes))
        self.w -= h

    def stochastic_descent(self, X, y):
        indexes = np.random.choice(X.shape[0], int(X.shape[0] * self.delta))
        self.w -= self.gradient_descent(np.take(X, indexes, axis=0), np.take(y, indexes))

    def gradient_descent(self, X, y):
        return self.eta * self.calc_gradient(X, y)

    def adagard_descent(self, X, y):
        pass

    def predict(self, X):
        if self.w is None:
            raise Exception('Not trained yet')

        return np.dot(X, self.w)

    def calc_gradient(self, X, y):
        """
        X: np.array of shape (l, d) (l can be equal to 1 if stochastic)
        y: np.array of shape (l)
        ---
        output: np.array of shape (d)
        """
        return 2 * np.dot(X.T, np.dot(X, self.w) - y) / y.shape[0]

    def calc_loss(self, X, y):
        """
        X: np.array of shape (l, d)
        y: np.array of shape (l)
        ---
        output: float
        """
        return np.mean((self.predict(X) - y) ** 2)


#### 7. [1 points] Train and validate "hand-written" models on the same data, and compare the quality with the Sklearn or StatsModels methods. Investigate the effect of the `max_iter` and `alpha` parameters on the optimization process. Is it consistent with your expectations?

In [23]:
features_train_last = features_train
features_test_last = features_test
target_train_last = target_train
target_test_last = target_test

In [24]:
w0 = np.random.uniform(-1, 1, features_train_last.shape[1])

In [25]:
gradient = LinReg(gd_type='GradientDescent', w0=w0, eta=0.5, tolerance=1e-6, max_iter=100).fit(features_train_last, target_train_last)
stoch = LinReg(gd_type='StochasticDescent', w0=w0, eta=0.5, tolerance=1e-6, max_iter=100).fit(features_train_last, target_train_last)
momentum = LinReg(gd_type='Momentum', eta=0.5, w0=w0, tolerance=1e-6, max_iter=100, alpha=0.1).fit(features_train_last, target_train_last)

In [26]:
def display_info_linear():
    predicted_test_last, predicted_train = calculations()
    print('Linear regression:')
    print(f'RMSE : {mean_squared_error(target_train, predicted_train, squared=False):.4f}')
    print(f'R2   : {r2_score(target_train, predicted_train):.4f}')
    print(f'RMSE test: {mean_squared_error(target_test, predicted_test_last, squared=False):.4f}')
    print(f'R2 test: {r2_score(target_test, predicted_test_last):.4f}')


def calculations():
    predicted_train = linear_results.predict(features_train)
    predicted_test_last = linear_results.predict(features_test)
    return predicted_test_last, predicted_train


display_info_linear()



Linear regression:
RMSE : 882.4742
R2   : 0.9137
RMSE test: 880.9212
R2 test: 0.9148


#### 8. [1 points] Plot graphs (on the same picture) of the dependence of the loss function value on the iteration number for Full GD, SGD, Momentum and Adagrad. Draw conclusions about the rate of convergence of various modifications of gradient descent.

Don't forget about what *beautiful* graphics should look like!